In [4]:
from my_secrets import API_KEY
import urllib.request
import json
import pandas as pd
import numpy as np
import sqlite3
import time
from loguru import logger

RESOURCE_ID = "f2e5503e-927d-4ad3-9500-4ab9e55deb59"

DB_NAME = "warsaw_ztm.db"
TABLE_NAME = "vehicles"

URL_BUS= f'https://api.um.warszawa.pl/api/action/busestrams_get/?resource_id=f2e5503e927d-4ad3-9500-4ab9e55deb59&apikey={API_KEY}&type=1'
URL_TRAM = f'https://api.um.warszawa.pl/api/action/busestrams_get/?resource_id=f2e5503e927d-4ad3-9500-4ab9e55deb59&apikey={API_KEY}&type=2'

In [ ]:
def create_db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            line TEXT,
            lon REAL,
            lat REAL,
            time TEXT,
            vehicle_number TEXT,
            PRIMARY KEY (vehicle_number, time)
        )
    ''')
    
    conn.commit()
    conn.close()

def fetch_data(url):
    with urllib.request.urlopen(url) as response:
        data = response.read().decode('utf-8')
        return json.loads(data)

def insert_data(data):
    if "Błędna" in data.get("result"):
        logger.error("Failed to connect to API.")
        return None
    
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    for item in data.get("result", []):        
        try:
            cursor.execute(f'''
                INSERT OR IGNORE INTO {TABLE_NAME} (line, lon, lat, time, vehicle_number)
                VALUES (?, ?, ?, ?, ?)
            ''', (
                item.get("Lines"),
                item.get("Lon"),
                item.get("Lat"),
                item.get("Time"),
                item.get("VehicleNumber")
            ))
        except Exception as e:
            logger.error(f"Failed to insert data: {e}")

    conn.commit()
    conn.close()

def main():
    create_db()
    error_counter = 0
    
    while True:
        try:            
            json_data = fetch_data(URL_BUS)
            insert_data(json_data)
            
            json_data = fetch_data(URL_TRAM)
            insert_data(json_data)
            
            time.sleep(30)
        
        except Exception as e:
            error_counter += 1
            logger.error(f"Error: {e}, waiting 10 seconds and trying again. There was {error_counter} errors so far.")
            time.sleep(4)

if __name__ == "__main__":
    main()

NameError: name 'sqlite3' is not defined

In [23]:
temp = fetch_data(f"https://api.um.warszawa.pl/api/action/busestrams_get/?resource_id=f2e5503e927d-4ad3-9500-4ab9e55deb59&apikey={API_KEY}&type=1")
for item in temp.get("result", []):
    print(item)

{'Lines': '161', 'Lon': 21.115819, 'VehicleNumber': '1000', 'Time': '2025-02-15 23:54:32', 'Lat': 52.234551, 'Brigade': '1'}
{'Lines': '161', 'Lon': 21.115668, 'VehicleNumber': '1001', 'Time': '2025-02-15 20:36:04', 'Lat': 52.234538, 'Brigade': '3'}
{'Lines': '161', 'Lon': 21.227478, 'VehicleNumber': '1002', 'Time': '2025-02-16 10:01:45', 'Lat': 52.18211, 'Brigade': '3'}
{'Lines': '219', 'Lon': 21.102446, 'VehicleNumber': '1003', 'Time': '2025-02-16 10:01:48', 'Lat': 52.222748, 'Brigade': '2'}
{'Lines': '119', 'Lon': 21.115081, 'VehicleNumber': '1004', 'Time': '2025-02-15 23:49:40', 'Lat': 52.234638, 'Brigade': '3'}
{'Lines': '161', 'Lon': 21.22471, 'VehicleNumber': '1005', 'Time': '2025-02-16 10:01:58', 'Lat': 52.186705, 'Brigade': '1'}
{'Lines': '219', 'Lon': 21.115911, 'VehicleNumber': '1006', 'Time': '2025-02-15 02:56:05', 'Lat': 52.234418, 'Brigade': '1'}
{'Lines': '119', 'Lon': 21.205586, 'VehicleNumber': '1007', 'Time': '2025-02-16 10:01:51', 'Lat': 52.210144, 'Brigade': '4'}
{'

In [7]:
temp = fetch_data(URL_BUS)
for item in temp.get("result", []):
    print(item)

{'Lines': '161', 'Lon': 21.210673, 'VehicleNumber': '1000', 'Time': '2025-02-15 20:50:15', 'Lat': 52.16087, 'Brigade': '1'}
{'Lines': '161', 'Lon': 21.115668, 'VehicleNumber': '1001', 'Time': '2025-02-15 20:36:04', 'Lat': 52.234538, 'Brigade': '3'}
{'Lines': '161', 'Lon': 21.219306, 'VehicleNumber': '1002', 'Time': '2025-02-15 20:50:17', 'Lat': 52.185393, 'Brigade': '2'}
{'Lines': '219', 'Lon': 21.093001, 'VehicleNumber': '1003', 'Time': '2025-02-15 20:50:15', 'Lat': 52.221654, 'Brigade': '2'}
{'Lines': '119', 'Lon': 21.000295, 'VehicleNumber': '1004', 'Time': '2025-02-15 20:50:26', 'Lat': 52.206123, 'Brigade': '3'}
{'Lines': '119', 'Lon': 21.205616, 'VehicleNumber': '1005', 'Time': '2025-02-15 20:50:25', 'Lat': 52.210106, 'Brigade': '5'}
{'Lines': '219', 'Lon': 21.115911, 'VehicleNumber': '1006', 'Time': '2025-02-15 02:56:05', 'Lat': 52.234418, 'Brigade': '1'}
{'Lines': '305', 'Lon': 21.115488, 'VehicleNumber': '1007', 'Time': '2025-02-14 21:02:48', 'Lat': 52.234616, 'Brigade': '1'}
{

In [ ]:
for item in data.get("result", []):